# LOAD DATA

In [1]:
import pandas as pd 

In [ ]:
df_ = pd.read_csv("./online_retail_II_Germany.csv")
df = df_.copy()

In [14]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536527,22809,SET OF 6 T-LIGHTS SANTA,6,2010-12-01 13:04:00,2.95,12662.0,Germany
1,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,2010-12-01 13:04:00,2.55,12662.0,Germany
2,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,2010-12-01 13:04:00,0.85,12662.0,Germany
3,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,2010-12-01 13:04:00,1.65,12662.0,Germany
4,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,2010-12-01 13:04:00,1.95,12662.0,Germany


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9495 entries, 0 to 9494
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      9495 non-null   object 
 1   StockCode    9495 non-null   object 
 2   Description  9495 non-null   object 
 3   Quantity     9495 non-null   int64  
 4   InvoiceDate  9495 non-null   object 
 5   Price        9495 non-null   float64
 6   Customer ID  9495 non-null   float64
 7   Country      9495 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 593.6+ KB


# PREPROCESSING

In [16]:
df_Invoice = pd.DataFrame({"Invoice":[row for row in df["Invoice"].values if "C"  not in str(row)]})
df_Invoice.head()
# trich xuat unique Invoice value without "C" value
df_Invoice = df_Invoice.drop_duplicates("Invoice")

In [17]:
# The transactions except cancelled transactions:
df = df.merge(df_Invoice, on = "Invoice")

In [18]:
# Outlier değerler için baskılama yapılacak low ve up limit belirleyelim:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


# Replace outliers with thresholds
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
  

In [19]:
df.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

In [20]:
num_cols = [col for col in df.columns if df[col].dtypes in ["int64","float64"] and "ID" not in col]

print(num_cols)

['Quantity', 'Price']


In [21]:
for col in num_cols:
    replace_with_thresholds(df, col)

/var/folders/pm/vdvchn5d3psc1n3ny2r3hpwr0000gn/T/ipykernel_26828/3563397573.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-117.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit


In [22]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,9042.0,13.068514,15.415909,1.0,6.00,10.00,12.00,198.500
Price,9042.0,3.395409,4.330312,0.0,1.25,1.95,3.75,44.685
Customer ID,9042.0,12649.005087,312.779872,12426.0,12481.00,12600.00,12662.00,14335.000


In [23]:
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

In [24]:
# Unique Number of Products (with Description)
print(df.Description.nunique())

# Unique Number of Products (with StockCode)
print(df.StockCode.nunique())

1696
1664


In [25]:
df_product = df[["Description","StockCode"]].drop_duplicates()
df_product = df_product.groupby(["Description"]).agg({"StockCode":"count"}).reset_index()
df_product.sort_values("StockCode", ascending=False).head()

,Description,StockCode
0,50'S CHRISTMAS GIFT BAG LARGE,1
1139,RED RETROSPOT MUG,1
1137,RED RETROSPOT LUGGAGE TAG,1
1136,RED RETROSPOT CUP,1
1135,RED RETROSPOT CHILDRENS UMBRELLA,1


In [26]:
df_product.rename(columns={'StockCode':'StockCode_Count'},inplace=True)

df_product = df_product[df_product["StockCode_Count"]>1]

In [27]:
df = df[~df["Description"].isin(df_product["Description"])]

In [28]:
print(df.StockCode.nunique())
print(df.Description.nunique())

1664
1696


In [29]:
# 2nd Step
df_product = df[["Description","StockCode"]].drop_duplicates()
df_product = df_product.groupby(["StockCode"]).agg({"Description":"count"}).reset_index()
df_product.rename(columns={'Description':'Description_Count'},inplace=True)

In [30]:
df_product = df_product.sort_values("Description_Count", ascending=False)
df_product.head()

,StockCode,Description_Count
991,22939,2
1586,84997C,2
685,22466,2
180,21232,2
517,22179,2


In [31]:
df_product = df_product[df_product["Description_Count"] > 1] 

df_product.head()

,StockCode,Description_Count
991,22939,2
1586,84997C,2
685,22466,2
180,21232,2
517,22179,2


In [32]:
df = df[~df["StockCode"].isin(df_product["StockCode"])]

In [58]:
# Now each stock code represents a single product:
print(df.StockCode.nunique())
print(df.Description.nunique())

1631
1631


In [34]:
df = df[~df["StockCode"].str.contains("POST", na=False)]

In [35]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

In [36]:
gr_inv_pro_df = create_invoice_product_df(df, id=True)
gr_inv_pro_df.head()

/var/folders/pm/vdvchn5d3psc1n3ny2r3hpwr0000gn/T/ipykernel_26828/3019334217.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  applymap(lambda x: 1 if x > 0 else 0)


StockCode,10002,10125,10135,11001,15034,15036,15039,15044A,15044B,15044D,...,90161D,90170,90173,90201A,90201B,90201C,90201D,90202D,90204,M
Invoice,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
def check_id(dataframe, stockcode):
    # Ensure that the stockcode passed and the StockCode column are of the same type (e.g., string or integer)
    dataframe["StockCode"] = dataframe["StockCode"].astype(str)  # Convert StockCode to string
    stockcode = str(stockcode)  # Convert the input stockcode to string as well

    filtered_df = dataframe[dataframe["StockCode"] == stockcode]["Description"].unique()
    
    if len(filtered_df) > 0:
        product_name = filtered_df[0]
        return stockcode, product_name
    else:
        return stockcode, "Product not found"


In [64]:
check_id(df,22809)

('22809', 'SET OF 6 T-LIGHTS SANTA')

In [65]:
# # save preprocessed data as a new csv file
# df.to_csv("online_retail_II_Germany_preprocessed.csv", index=False)